In [ ]:
!pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html


Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 1156.7 MB 12.1 MB/s eta 0:00:01    |███████▌                        | 271.1 MB 11.4 MB/s eta 0:01:19     |████████████▉                   | 464.3 MB 3.0 MB/s eta 0:03:50         | 488.3 MB 3.3 MB/s eta 0:03:22��███████████▋                | 565.3 MB 307 kB/s eta 0:32:06/s eta 0:00:54███████▋              | 635.0 MB 11.8 MB/s eta 0:00:45��█████████████▋              | 635.5 MB 11.8 MB/s eta 0:00:45████████              | 647.8 MB 11.2 MB/s eta 0:00:460:360 MB 9.9 MB/s eta 0:00:41        | 822.1 MB 10.4 MB/s eta 0:00:33        | 824.7 MB 10.4 MB/s eta 0:00:32█████████████         | 828.5 MB 10.4 MB/s eta 0:00:32█████▊        | 856.7 MB 1.3 MB/s eta 0:03:44     |████████████████████████        | 866.8 MB 1.4 MB/s eta 0:03:34██████▏       | 872.6 MB 1.4 MB/s eta 0:03:30    |████████████████████████▋       | 889.6 MB 1.4 MB/s eta 0:03:16�████▊       | 893.7 MB 1.4 MB/s eta 0:03:13   

In [4]:
import torch, tqdm, math, re, string,random
import numpy as np
import sklearn.metrics as metrics
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import torch.nn as nn
from torch.nn import CrossEntropyLoss, CosineSimilarity
from torch.utils.data import DataLoader, SequentialSampler, TensorDataset, Dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [5]:
device = torch.device('cuda:1')

def sent_pred(sent):
    encoded_dict = tokenizer(sent,add_special_tokens = True,max_length = 256,
                                         return_attention_mask = True,return_tensors = 'pt', truncation=True)   
    iids = encoded_dict['input_ids'].to(device)
    amasks = encoded_dict['attention_mask'].to(device)
    p = bert(iids, token_type_ids=None, attention_mask=amasks)[0]
    return torch.argmax(p).item()

def sentences_pred(sents, batch_size=32):
    encoded_dict = tokenizer(sents,add_special_tokens = True, max_length = 256, padding='max_length', 
                             return_attention_mask = True, return_tensors = 'pt',truncation=True)   
    bert.to(device)
    bert.eval()
    input_ids = encoded_dict['input_ids'].to(device)
    input_mask = encoded_dict['attention_mask'].to(device)
    with torch.no_grad():
        logits = bert(input_ids, token_type_ids=None, attention_mask=input_mask)[0]
        probs = nn.functional.softmax(logits, dim=-1)
    return probs
    return torch.cat(probs_all, dim=0)


def importscore(text, sample=0):
    random.seed(2020)
    text_ls = text.split()
    text_ls = text_ls[:200]
#     no_text_ls = [(i,j) for i,j in enumerate(text_ls)]
    len_text = len(text_ls)
    if 0<sample<len_text:
        leave_1_texts = [' '.join(text_ls[:ii] + text_ls[min(ii + 1, len_text):]) 
                         for ii in random.sample(list(range(len_text)),sample)]
    else:
        leave_1_texts = [' '.join(text_ls[:ii] + text_ls[min(ii + 1, len_text):]) for ii in range(len_text)]
    leave_1_probs = sentences_pred([text]+leave_1_texts)
    orig_probs = leave_1_probs[:1].squeeze()
    orig_label = torch.argmax(orig_probs)
    orig_prob = orig_probs.max()
    leave_1_probs_argmax = torch.argmax(leave_1_probs[1:], dim=-1)
    import_scores = (orig_prob - leave_1_probs[1:, orig_label]).data.cpu().numpy()
    return import_scores

def cal_AUC(orig_sent, adv_sent, k=0):
    nms, nms1 = [], []
    for i in tqdm.tqdm(range(int(len(orig_sent)))):
        isa = importscore(adv_sent[i],k)
        iso = importscore(orig_sent[i],k)
        nms.append({'idx': i, 'isa': isa, 'iso': iso})
    for i in nms:
        isa_ = sorted(i['isa'],reverse=True)
        iso_ = sorted(i['iso'],reverse=True)
        entropyo = -np.mean([abs(i)*np.log(abs(i)) for i in iso_[:] if i!=0])
        entropya = -np.mean([abs(i)*np.log(abs(i)) for i in isa_[:] if i!=0])
    #     stdo, stda = np.std(iso_[:128]), np.std(isa_[:128])
        nms1.append({'idx': i['idx'], 'entropyo': entropyo, 'entropya':entropya})
    preds = [i['entropyo']for i in nms1]+[i['entropya']for i in nms1]
    y_test = [0]*len(nms1)+[1]*len(nms1)
    y_test = label_binarize(y_test, classes=[0, 1])
    fpr, tpr, threshold = metrics.roc_curve(y_test,preds)
    roc_auc = metrics.auc(fpr, tpr)
    return roc_auc
    
def clean_str(string, TREC=False):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Every dataset is lower cased except for TREC
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip() if TREC else string.strip().lower()

In [3]:
import sys
sys.path.append("..")
from model.sequence_classification import (
    BertPrefixForSequenceClassification,
    BertPromptForSequenceClassification,
    RobertaPrefixForSequenceClassification,
    RobertaPromptForSequenceClassification
)

In [85]:
# bert = AutoModelForSequenceClassification.from_pretrained('../checkpoints/amazon-bert-normal')
# tokenizer = AutoTokenizer.from_pretrained('../checkpoints/amazon-bert-normal', do_lower_case=True);
# bert = BertPrefixForSequenceClassification.from_pretrained('../checkpoints/amazon-bert-prefix')
# tokenizer = AutoTokenizer.from_pretrained('../checkpoints/amazon-bert-prefix', do_lower_case=True);
bert = BertPromptForSequenceClassification.from_pretrained('../checkpoints/amazon-bert-prompt')
tokenizer = AutoTokenizer.from_pretrained('../checkpoints/amazon-bert-prompt', do_lower_case=True);

In [90]:
f = open('adv_output/amazon-bert-prompt/textbugger/2022-07-02-21-53-log.txt')
txt = f.read()
text = txt.split('--------------------------------------------- Result ')
for i in text[-1].split('\n')[-10:-1]:
    print(i)

Number of successful attacks: 313
Number of failed attacks: 140
Number of skipped attacks: 47
Original accuracy: 90.6%
Accuracy under attack: 28.0%
Attack success rate: 69.09%
Average perturbed word %: 24.91%
Average num. words per input: 82.49
Avg num queries: 206.54


In [91]:
orig_sent_, adv_sent_, nms, nms2 = [], [], [], []
count = 0
for i in range(1,501):
    tmp0 = text[i].split('\n')
    if 'FAILED' in tmp0[1] or 'SKIPPED' in tmp0[1]:
        pass
    else:
        orig_sent_.append(tmp0[3])
        adv_sent_.append(tmp0[5])
orig_sent = [i.replace('[','').replace(']','') for i in orig_sent_]
adv_sent = [i.replace('[','').replace(']','') for i in adv_sent_]

In [92]:
for i in tqdm.tqdm(range(int(len(orig_sent)))):
    isa = importscore(adv_sent[i],20)
    iso = importscore(orig_sent[i],20)
    nms.append({'idx': i, 'isa': isa, 'iso': iso})
for i in tqdm.tqdm(nms):
    isa_ = sorted(i['isa'],reverse=True)
    iso_ = sorted(i['iso'],reverse=True)
    entropyo = -np.mean([abs(i)*np.log(abs(i)) for i in iso_[:] if i!=0])
    entropya = -np.mean([abs(i)*np.log(abs(i)) for i in isa_[:] if i!=0])
#     stdo, stda = np.std(iso_[:128]), np.std(isa_[:128])
    nms2.append({'idx': i['idx'], 'entropyo': entropyo, 'entropya':entropya})

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 313/313 [00:00<00:00, 5251.60it/s]


In [93]:
preds = [i['entropyo']for i in nms2]+[i['entropya']for i in nms2]
y_test = [0]*len(nms2)+[1]*len(nms2)
y_test = label_binarize(y_test, classes=[0, 1])
fpr, tpr, threshold = metrics.roc_curve(y_test,preds)
roc_auc = metrics.auc(fpr, tpr)
roc_auc

0.8844124161724627

In [94]:
f = open('adv_output/amazon-bert-prompt/textfooler/2022-07-02-22-26-log.txt')
txt = f.read()
text = txt.split('--------------------------------------------- Result ')
for i in text[-1].split('\n')[-10:-1]:
    print(i)

Number of successful attacks: 352
Number of failed attacks: 101
Number of skipped attacks: 47
Original accuracy: 90.6%
Accuracy under attack: 20.2%
Attack success rate: 77.7%
Average perturbed word %: 8.89%
Average num. words per input: 82.49
Avg num queries: 295.5


In [95]:
orig_sent_, adv_sent_, nms, nms2 = [], [], [], []
count = 0
for i in range(1,501):
    tmp0 = text[i].split('\n')
    if 'FAILED' in tmp0[1] or 'SKIPPED' in tmp0[1]:
        pass
    else:
        orig_sent_.append(tmp0[3])
        adv_sent_.append(tmp0[5])
orig_sent = [i.replace('[','').replace(']','') for i in orig_sent_]
adv_sent = [i.replace('[','').replace(']','') for i in adv_sent_]

In [96]:
for i in tqdm.tqdm(range(int(len(orig_sent)))):
    isa = importscore(adv_sent[i],20)
    iso = importscore(orig_sent[i],20)
    nms.append({'idx': i, 'isa': isa, 'iso': iso})
for i in tqdm.tqdm(nms):
    isa_ = sorted(i['isa'],reverse=True)
    iso_ = sorted(i['iso'],reverse=True)
    entropyo = -np.mean([abs(i)*np.log(abs(i)) for i in iso_[:] if i!=0])
    entropya = -np.mean([abs(i)*np.log(abs(i)) for i in isa_[:] if i!=0])
#     stdo, stda = np.std(iso_[:128]), np.std(isa_[:128])
    nms2.append({'idx': i['idx'], 'entropyo': entropyo, 'entropya':entropya})

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 352/352 [00:00<00:00, 5614.67it/s]


In [97]:
preds = [i['entropyo']for i in nms2]+[i['entropya']for i in nms2]
y_test = [0]*len(nms2)+[1]*len(nms2)
y_test = label_binarize(y_test, classes=[0, 1])
fpr, tpr, threshold = metrics.roc_curve(y_test,preds)
roc_auc = metrics.auc(fpr, tpr)
roc_auc

0.8870092975206612